# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re

import nltk
# download libraries if necessary
# nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [4]:
# load data from database
engine = create_engine('sqlite:///CleanDatabase.db')
df = pd.read_sql_table('cleaned_data', engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    """ Function to process text in the following order: normalize, tokenize, 
        remove stop words, and lemmatize.
    Args:
        text (str): text to be processed
    Returns:
        tokens (list): list of tokens after processing text
    """

    # normalize text by removing punctuation and converting to lower case
    text = re.sub(r'[^\w\s]', ' ', text.lower())

    # tokenize text
    tokens = word_tokenize(text)

    # remove stop words and lemmatize
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word)
              for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier

In [19]:
# build pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [149]:
from sklearn.model_selection import train_test_split

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001F72C0BB1F8>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [150]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [151]:
# convert y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns

In [153]:
from sklearn.metrics import classification_report

In [155]:
# test function
def get_report(y_test, y_pred):
    """ Function that compares the predicted labels to the test labels returning the mean accuracy, 
        f1-score macro average and f1-score weighted average of the model.
    Args:
        y_test (DataFrame): test labels
        y_pred (DataFrame): predicted labels
    Returns:
        (DataFrame): mean accuracy, f1-score macro average and f1-score weighted average of labels
    """

    results = []
    for column in y_test.columns:
        report = classification_report(
            y_test[column], y_pred[column], output_dict=True)
        results.append([report['accuracy'], report['macro avg']
                        ['f1-score'], report['weighted avg']['f1-score']])

    index = y_test.columns
    columns = ['accuracy', 'f1-score macro avg', 'f1-score weighted avg']

    return pd.DataFrame(results, index=index, columns=columns).mean()

In [156]:
# check results
results = get_report(y_test, y_pred)
results

accuracy                 0.928883
f1-score macro avg       0.512088
f1-score weighted avg    0.900306
dtype: float64

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
from sklearn.model_selection import GridSearchCV

In [36]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'tfidf__sublinear_tf': (True, False),
    'clf__estimator__leaf_size': (20, 40),
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 1), score=0.229, total= 1.4min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s


[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 1), score=0.204, total= 1.3min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 1) 


...

[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 2), score=0.234, total= 1.4min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 12.4min remaining:    0.0s


[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 2), score=0.249, total= 1.4min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1), score=0.229, total= 1.3min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1), score=0.207, total= 1.3min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1), score=0.232, total= 1.4min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1), score=0.238, total= 1.4min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__n

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 75.9min finished


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001F6F40FF558>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=KNeighborsClassifier()))]),
             param_grid={'clf__estimator__leaf_size': (20, 40),
                         'tfidf__sublinear_tf': (True, False),
                         'vect__ngram_range': ((1, 1), (1, 2))},
             verbose=10)

In [39]:
# best parameters
cv.best_params_

{'clf__estimator__leaf_size': 20,
 'tfidf__sublinear_tf': False,
 'vect__ngram_range': (1, 1)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [157]:
# build pipeline using best parameters
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 1))),
    ('tfidf', TfidfTransformer(sublinear_tf=False)),
    ('clf', MultiOutputClassifier(KNeighborsClassifier(leaf_size=20)))
])

In [158]:
# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001F72C0BB1F8>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(leaf_size=20)))])

In [159]:
# predict on test data
y_pred = cv.predict(X_test)

# convert y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns

# check results
results = get_report(y_test, y_pred)
results

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy                 0.946828
f1-score macro avg       0.625059
f1-score weighted avg    0.935607
dtype: float64

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Random Forest Classifier

In [161]:
from sklearn.ensemble import RandomForestClassifier

In [162]:
# build pipeline using RandomForestClassifier
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [163]:
# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001F72C0BB1F8>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [164]:
# predict on test data
y_pred = pipeline.predict(X_test)

# convert y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns

# check results
results = get_report(y_test, y_pred)
results

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy                 0.948079
f1-score macro avg       0.620252
f1-score weighted avg    0.936225
dtype: float64

In [63]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__bootstrap': (True, False),
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__bootstrap=True, vect__ngram_range=(1, 1) ........


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__bootstrap=True, vect__ngram_range=(1, 1), score=0.273, total= 4.9min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.9min remaining:    0.0s


[CV] clf__estimator__bootstrap=True, vect__ngram_range=(1, 1) ........
[CV]  clf__estimator__bootstrap=True, vect__ngram_range=(1, 1), score=0.259, total= 6.8min


...

[CV] clf__estimator__bootstrap=True, vect__ngram_range=(1, 2) ........
[CV]  clf__estimator__bootstrap=True, vect__ngram_range=(1, 2), score=0.263, total=14.7min


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 104.8min remaining:    0.0s


[CV] clf__estimator__bootstrap=True, vect__ngram_range=(1, 2) ........
[CV]  clf__estimator__bootstrap=True, vect__ngram_range=(1, 2), score=0.276, total=16.3min
[CV] clf__estimator__bootstrap=False, vect__ngram_range=(1, 1) .......
[CV]  clf__estimator__bootstrap=False, vect__ngram_range=(1, 1), score=0.270, total= 7.5min
[CV] clf__estimator__bootstrap=False, vect__ngram_range=(1, 1) .......
[CV]  clf__estimator__bootstrap=False, vect__ngram_range=(1, 1), score=0.263, total= 7.6min
[CV] clf__estimator__bootstrap=False, vect__ngram_range=(1, 1) .......
[CV]  clf__estimator__bootstrap=False, vect__ngram_range=(1, 1), score=0.267, total= 7.6min
[CV] clf__estimator__bootstrap=False, vect__ngram_range=(1, 1) .......
[CV]  clf__estimator__bootstrap=False, vect__ngram_range=(1, 1), score=0.260, total= 7.9min
[CV] clf__estimator__bootstrap=False, vect__ngram_range=(1, 1) .......
[CV]  clf__estimator__bootstrap=False, vect__ngram_range=(1, 1), score=0.275, total= 8.0min
[CV] clf__estimator__bo

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 295.9min finished


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001F6F40FF558>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__bootstrap': (True, False),
                         'vect__ngram_range': ((1, 1), (1, 2))},
             verbose=10)

In [64]:
# best parameters
cv.best_params_

{'clf__estimator__bootstrap': False, 'vect__ngram_range': (1, 2)}

In [166]:
# build pipeline using best parameters
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(bootstrap=False)))
])

In [167]:
# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function tokenize at 0x000001F72C0BB1F8>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=False)))])

In [169]:
# predict on test data
y_pred = pipeline.predict(X_test)

# convert y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns

# check results
results = get_report(y_test, y_pred)
results

accuracy                 0.947037
f1-score macro avg       0.625294
f1-score weighted avg    0.935779
dtype: float64

#### Use custom transformers

In [170]:
from sklearn.base import BaseEstimator, TransformerMixin

In [171]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        """ Function that takes in a text message string and returns True if the first word is a verb.
        Args:
            text (str): string containing text message
        Returns:
            (bool): True if first word of sentence is a verb
        """
        sentence_list = nltk.sent_tokenize(text)
        lemmatizer = WordNetLemmatizer()
        for sentence in sentence_list:
            tokens = word_tokenize(sentence)
            clean_tokens = []
            for token in tokens:
                clean_tokens.append(
                    lemmatizer.lemmatize(token).lower().strip())
            pos_tags = nltk.pos_tag(clean_tokens)
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP']:
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        """ Function that takes in a series of messages and returns a dataframe with starting_verb() applied
            to each message.
        Args:
            X (Series): list of messages
        Returns:
            (DataFrame): boolean values indicating if first word in the messages is a verb
        """
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [172]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        """ Function that takes in a series of messages and returns a dataframe containing the message lengths.
        Args:
            X (Series): list of messages
        Returns:
            (DataFrame): integer values representing message lengths
        """
        X_len = pd.Series(X).apply(len)
        return pd.DataFrame(X_len)

In [173]:
from sklearn.pipeline import FeatureUnion

In [174]:
# build pipeline with both transformers
pipeline = Pipeline([
    ('feature', FeatureUnion([
        ('nlp_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2))),
            ('tfidf', TfidfTransformer())
        ])),
        ('sve', StartingVerbExtractor()),
        ('tle', TextLengthExtractor())
    ])
    ),
    ('clf', MultiOutputClassifier(RandomForestClassifier(bootstrap=False)))
])

In [175]:
# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('feature',
                 FeatureUnion(transformer_list=[('nlp_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(ngram_range=(1,
                                                                                               2),
                                                                                  tokenizer=<function tokenize at 0x000001F72C0BB1F8>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('sve',
                                                 StartingVerbExtractor()),
                                                ('tle',
                                                 TextLengthExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimato

In [176]:
# predict on test data
y_pred = pipeline.predict(X_test)

# convert y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns

# check results
results = get_report(y_test, y_pred)
results

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy                 0.947093
f1-score macro avg       0.623521
f1-score weighted avg    0.935540
dtype: float64

#### StartingVerbExtractor

In [177]:
# build pipeline with only StartingVerbExtractor
pipeline = Pipeline([
    ('feature', FeatureUnion([
        ('nlp_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2))),
            ('tfidf', TfidfTransformer())
        ])),
        ('sve', StartingVerbExtractor())
    ])
    ),
    ('clf', MultiOutputClassifier(RandomForestClassifier(bootstrap=False)))
])

In [178]:
# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('feature',
                 FeatureUnion(transformer_list=[('nlp_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(ngram_range=(1,
                                                                                               2),
                                                                                  tokenizer=<function tokenize at 0x000001F72C0BB1F8>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('sve',
                                                 StartingVerbExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=False)))])

In [179]:
# predict on test data
y_pred = pipeline.predict(X_test)

# convert y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns

# check results
results = get_report(y_test, y_pred)
results

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy                 0.946867
f1-score macro avg       0.624798
f1-score weighted avg    0.935581
dtype: float64

#### TextLengthExtractor

In [180]:
# build pipeline with only TextLengthExtractor
pipeline = Pipeline([
    ('feature', FeatureUnion([
        ('nlp_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2))),
            ('tfidf', TfidfTransformer())
        ])),
        ('tle', TextLengthExtractor())
    ])
    ),
    ('clf', MultiOutputClassifier(RandomForestClassifier(bootstrap=False)))
])

In [181]:
# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('feature',
                 FeatureUnion(transformer_list=[('nlp_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(ngram_range=(1,
                                                                                               2),
                                                                                  tokenizer=<function tokenize at 0x000001F72C0BB1F8>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('tle',
                                                 TextLengthExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=False)))])

In [182]:
# predict on test data
y_pred = pipeline.predict(X_test)

# convert y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns

# check results
results = get_report(y_test, y_pred)
results

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy                 0.946920
f1-score macro avg       0.623124
f1-score weighted avg    0.935304
dtype: float64

The custom features added do not improve the model, hence we will choose the model having best parameters for the random forest classifier.

### 9. Export your model as a pickle file

In [183]:
# build pipeline using best parameters
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(bootstrap=False)))
])

# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function tokenize at 0x000001F72C0BB1F8>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=False)))])

In [184]:
# export model as pickle file
import pickle

pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(pipeline, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.